# Aplicando 4

## Pergunta 1

In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import requests
import xmltodict

In [16]:
# Definicao dos das URLs das APIs desejadas
url_universidades_all = "http://universities.hipolabs.com/search"
url_universidades_us = "http://universities.hipolabs.com/search?country=United%20States"
url_twitter = "https://api.twitter.com/2/tweets/search/recent"
url_musicos = "http://musicbrainz.org/ws/2/artist/?query=country:US&limit=80"

In [8]:
## API de universidades (JSON)

# Extracao de dados de uma API em JSON diretamente pela funcao read_json
df_universidades_us = pd.read_json(url_universidades_us)
df_universidades_us.shape

(2300, 6)

## Pergunta 2

In [11]:
df_universidades_us.sample(5)

,alpha_two_code,web_pages,state-province,name,domains,country
1363,US,[http://www.lahc.edu],None,Los Angeles Harbor College,[lahc.edu],United States
1231,US,[http://www.xula.edu/],None,Xavier University of Louisiana,[xula.edu],United States
1848,US,[http://www.durhamtech.edu],None,Durham Technical Community College,[durhamtech.edu],United States
47,US,[https://itu.edu/],None,International Technological University,[itu.edu],United States
567,US,[http://www.luthersem.edu/],None,Luther Seminary,[luthersem.edu],United States


## Pergunta 3

In [25]:
# Chamada da API (GET)
response_api = requests.request("GET", url_musicos)

# Extracao dos dados em XML e conversao para dicionario
dict_data = xmltodict.parse(response_api.content)

# Filtragem de dados especificos (artist)
dict_data_artist = dict_data["metadata"]["artist-list"]["artist"]

# Extracao dos dados dos artisitas (lista)
l_base_musicos = []
for artist in dict_data_artist:
    
    # Concatenacao da lista de tags, separadas por ";"
    if "tag-list" in artist.keys():
        tags_base = ""
        if type(artist["tag-list"]["tag"]) is list:
            for tag in artist["tag-list"]["tag"]:
                tags_base = tags_base + tag.get("name", "") + ";"
        else:
           tags_base = artist.get("tag-list").get("tag").get("name", np.NaN)
    else:
        tags_base = np.NaN
            
    l_base_musicos.append(
       {
           "id": artist.get("@id", np.NaN),
           "ipi": artist["ipi-list"].get("ipi", np.NaN) if artist.get("ipi-list", False) else np.NaN,
           "name": artist.get("name", np.NaN),
           "type": artist.get("@type", np.NaN),
           "tags": tags_base
       }
   )

# Conversao de lista para DataFrame pandas
df_musicos = pd.DataFrame(l_base_musicos)


In [26]:
df_musicos.shape

(80, 5)

## Pergunta 4

In [27]:
df_musicos.sample(5)

,id,ipi,name,type,tags
16,ebfc1398-8d96-47e3-82c3-f782abcdb13d,NaN,The Beach Boys,Group,rock;pop;american;usa;psychedelic rock;américa...
17,070d193a-845c-479f-980e-bef15710653e,"[00045620792, 00052210040, 00334284961]",Prince,Person,rock;pop;70s;80s;90s;dance;punk;american;usa;f...
5,6faa7ca7-0d99-4a5e-bfa6-1fd5037520c6,NaN,Grateful Dead,Group,rock;folk;american;blues;folk rock;rnb;psyched...
11,e01646f2-2a04-450d-8bf2-0d993082e058,NaN,Phish,Group,rock;alternative rock;progressive rock;jazz;am...
47,0bbbc496-c7b5-4b3f-bb6d-bd312827d6e5,00022974973,Roy Orbison,Person,rock;pop;american;rockabilly;baroque pop;pop r...


## Pergunta 5

In [29]:
# Importar a base stores
df_stores = pd.read_csv("./stores.csv")

In [31]:
df_stores.agg(
    {
        "Items_Available": ["mean", "var", "std"]
    }
)

,Items_Available
mean,1782.035714
var,89923.247885
std,299.872053


## Pergunta 6

In [34]:
df_stores["Items_Available"].quantile([.25, .5, .75])

0.25    1575.50
0.50    1773.50
0.75    1982.75
Name: Items_Available, dtype: float64

## Pergunta 7

In [36]:
# Importar a base best_selling_mobile_phones
df_mobile = pd.read_csv("./best_selling_mobile_phones.csv")

In [39]:
df_mobile.groupby('year')['units_sold_m'].mean()

year
1996     60.000000
1999    161.000000
2000    126.000000
2003     77.571429
2004     73.333333
2005     99.500000
2006     26.333333
2007     44.125000
2008     17.500000
2009     60.000000
2010     58.333333
2011     27.800000
2012     64.050000
2013     89.500000
2014     79.500000
2015    109.550000
2016     38.142857
2017     51.140000
2018     33.950000
2019     24.788889
2020     20.380000
2021     13.500000
Name: units_sold_m, dtype: float64

In [41]:
df_mobile.groupby('form')['units_sold_m'].median().sort_values(ascending=False)

form
Bar             65.0
Flip phone      47.5
Touchscreen     18.2
Slider          18.0
Keyboard bar     8.5
Taco             3.0
Tilt slider      2.0
Name: units_sold_m, dtype: float64